#### Business Analytics FHDW 2024
# Dimensionsreduktion mit der Principal Components Analysis
## am Beispiel von Daten über Frühstücksflocken

Das Beispiel ist aus Shmueli et al.; für unsere Veranstaltung und Zwecke ein wenig umgebaut und aufbereitet. Wir brauchen neben den schon vorgestellten Hilfsmitteln und einer kleinen Auswertungsfunktion `pcaSummary` wieder den Datensatz *cereal.csv*, den wir in ein `DataFrame` einlesen.

In [ ]:
import pandas as pd
from sklearn.decomposition import PCA
from sklearn import preprocessing
import numpy as np

def pcaSummary(pca):
    return pd.DataFrame({'Varianz':pca.explained_variance_,
                         'Kumulierte Varianz':np.cumsum(pca.explained_variance_),
                         'Varianzanteil':pca.explained_variance_ratio_,
                         'Kumulierte Anteile':np.cumsum(pca.explained_variance_ratio_)})

cereals_df = pd.read_csv('./Daten/cereal.csv')
cereals_df

Zunächst interessieren uns nur zwei Variablen aus dem Datensatz: Kalorien (*calories*) und Bewertung (*rating*). Hängen die irgendwie zusammen? Die Korrelationsmatrix eines Dataframe kennen wir ja schon:

In [ ]:
cereals_df[['calories', 'rating']].corr()

Kurze Erinnerung: Nach unserer Vorlesung DAML ist das eine starke (negative) Korrelation zwischen den beiden Variablen.

$0.1 \leq |r| \lt 0.3 \to$ schwach

$0.3 \leq |r| \lt 0.5 \to$ moderat

$0.5 \leq |r| \leq 1 \to$ stark

Wenn also ein Produkt viele Kalorien hat, wird es niedrig bewertet. Hoch bewertete Produkte haben hingegen wenig Kalorien.

Für uns sind hier die Varianzen der Variablen eine Größe für ihren Informationsgehalt. Schauen wir uns die Kovarianzen der beiden Variablen an und setzen sie in Verhältnis:

In [ ]:
cereals_df[['calories', 'rating']].cov()

In [ ]:
variance_calories = cereals_df.calories.var()
variance_rating = cereals_df.rating.var()
variance_overall = variance_calories + variance_rating
 
print(f'Varianz Kalorien = {variance_calories:.4f}, Varianz Bewertung = {variance_rating:.4f}, Summe Varianz = {variance_overall:.4f}')
print(f'Verhältnis Varianz Kalorien / Summe Varianzen = {(variance_calories / variance_overall):.4f}')

Die Daten über Kalorien sind also für knapp 66% der Varianz insgesamt verantwortlich. Würden wir *rating* nun entfernen, gingen uns 34% der Information über die Varianz insgesamt verloren. Das ist viel, daher wäre unser händischer Ansatz aus dem Abschnitt *Korrelationsanalyse* hier nicht ideal. Werfen wir einen Blick auf die Daten, ihre Eigenvektoren und deren Eigenwerte. 

In [ ]:
plot_1 = cereals_df.plot.scatter(x='calories', y='rating')

cov_matrix = cereals_df[['calories', 'rating']].cov()
eigenvalues, eigenvectors = np.linalg.eig(cov_matrix)

print(f'Der erste Eigenvektor {eigenvectors[:,0]} hat den Eigenwert {eigenvalues[0]:.4f}')
print(f'Der zweite Eigenvektor {eigenvectors[:,1]} hat den Eigenwert {eigenvalues[1]:.4f}')

calories_mean = cereals_df.calories.mean()
rating_mean = cereals_df.rating.mean()
plot_1.quiver(calories_mean, rating_mean, *eigenvectors[:,0], color=['g'], scale=3)
plot_1.quiver(calories_mean, rating_mean, *eigenvectors[:,1], color=['r'], scale=5)

Unser zweidimensionaler Datensatz liefert uns aus seiner Kovarianzmatrix (s. o.) zwei *Eigenvektoren*. Deren Merkmale können wir hier ausnutzen. Denn wir suchen zu unseren ursprünglichen Variablen $X_{calories}$ und $X_{rating}$ neue Variablen $PC_{1}$ und $PC_{2}$, die *gewichtete* Durchschnitte der ursprünglichen Werte abzüglich des Mittelwertes sind, so dass $PC_{1}$ und $PC_{2}$ *unkorreliert* sind. Z. B. für $PC_{1} = a_{1,1}(X_{calories}-\overline{X}_{calories})+a_{1,2}(X_{rating}-\overline{X}_{rating})$. Dann sortieren wir diese Linearkombinationen $PC$ der ursprünglichen $X$ nach ihrer Varianz. 

Die dazu passenden Gewichte $a_{i,j}$ erhalten wir aus den Eigenvektoren. Im Diagramm oben liegt der erste (grüne) Eigenvektor auf der Richtung der stärksten Varianz und damit der höchsten Information, mit einem höheren Eigenwert von 498.02 als der zweite (rote) Eigenvektor mit 78.93. Der liegt auf der Richtung der zweitstärksten Varianz und ist dabei orthogonal zum bzw. linear unabhängig vom ersten Eigenvektor - anschaulich steht er senkrecht darauf. Tatsächlich repräsentieren die beiden Eigenvektoren die gesuchten *Principal Components* bzw. Hauptkomponenten. Mit ihnen projizieren wir unseren ursprünglichen Datenraum auf ein neues Koordinatensystem, in dem die einzelnen Dimensionen, für uns also praktisch die Variablen mit ihren Daten, linear unkorreliert sind, wie wir es benötigen.

Es gibt spezifische, effiziente Methoden der Implementierung einer PCA. Und mit zusätzlicher Hilfe von *scikit-learn* ist sie für unser Beispiel schnell durchgeführt:

In [ ]:
pca = PCA(n_components=2)
pca.fit(cereals_df[['calories', 'rating']])

Das `PCA`-Objekt kapselt für uns die einzelnen oben beschriebenen Eigen-Operationen, iteriert sie über die Variablenmenge mit ihren Daten und sortiert die Ergebnisse der Hauptkomponenten nach absteigender Varianz/Information. Schauen wir uns die Details an, indem wir die Resultate der PCA nach ihrer Ausführung etwas aufbereiten. Wir können verschiedene ihrer Elemente abfragen.

In [ ]:
pca_summary = pcaSummary(pca)
pca_summary = pca_summary.transpose()
pca_summary.columns = ['PC1', 'PC2']
print(pca_summary.to_string()+'\n')
pca_components = pd.DataFrame(pca.components_.transpose(),columns=['PC1', 'PC2'], index=['calories', 'rating'])
print(pca_components.to_string()+'\n')

Unsere Hauptkomponenten und damit die transformierten Daten besitzen nun jeweils eine Varianz von 498.02 und 78.93. Die Werte kommen uns bekannt vor: Es sind die Eigenwerte von oben. In Summe zeigt der transformierte Datensatz die gleiche Varianz von 576.96, wie das Original. Der Anteil der ersten Variable $PC1$ an der Gesamtvarianz beträgt nun aber über 86%, die zweite Variable steuert also nur noch etwas über 13% an Informationen bei.

Um die umgewandelten Daten auch nutzen zu können, projizieren wir den ursprünglichen Datensatz mit Hilfe der Vektoren in `pca.components_` auf das neue Koordinatensystem: 

In [ ]:
projected_values = pd.DataFrame(pca.transform(cereals_df[['calories', 'rating']]), columns=['PC1', 'PC2'])
projected_values

Überprüfen wir noch die Korrelation der neuen Variablen:

In [ ]:
projected_values.corr().round(2)

Gegenüber unserem Ausgangspunkt oben hätten wir uns verbessert, wenn wir uns nun auf die Betrachtung nur der Variable $PC1$ beschränken würden. Statt 34% würden nur noch 13% an Informationen unberücksichtigt bleiben.

Die Anwendung der PCA mit *scikit-learn* ist schnell implementiert. Also dehnen wir sie auf den gesamten Datensatz aus:

In [ ]:
pca_full = PCA()
# iloc entfernt hier die vorderen drei Spalten, dropna (*n*ot *a*vailable) Zeilen (axis=0) mit fehlenden Werten:
pca_full.fit(cereals_df.iloc[:, 3:].dropna(axis=0))

In [ ]:
pca_full_summary = pcaSummary(pca_full)
pca_full_summary = pca_full_summary.transpose()
pca_full_summary.columns = [f'PC{i}' for i in range(1, len(pca_full_summary.columns)+1)]
pca_full_summary.round(4)

Laut der Auswertung der PCA über alle 13 Komponenten decken die ersten drei bereits über 96% der Informationen ab. Zehn Variablen könnten wir also ohne Probleme unberücksichtigt lassen. Oder? Werfen wir einen Blick auf die Elemente der Eigenvektoren, also die Gewichte.

In [ ]:
pca_full_components = pd.DataFrame(pca_full.components_.transpose(),
                                   columns=pca_full_summary.columns,
                                   index=cereals_df.iloc[:, 3:].columns)
pca_full_components

Die Gewichte zeigen, wie stark jede der ursprünglichen Variablen die unterschiedlichen Hauptkomponenten beeinflusst. Das gibt uns einen Eindruck über die Struktur der untersuchten Daten.

Im gegebenen Fall sehen wir, dass die erste Hauptkomponente sehr stark durch den Anteil an Sodium im jeweiligen Produkt bestimmt wird, praktisch den Sodium-Gehalt misst. Die zweite Hauptkomponente misst dafür das Kalium (potassium). Hier müssen wir beachten, dass diese beiden Anteile in Milligramm, die anderen jedoch in Gramm angegeben werden. Entsprechend sind die Varianzen von Sodium und Kalium im Verhältnis sehr hoch und ihr Anteil an der Gesamtvarianz damit auch (vollziehen Sie das z. B. über `cereals_df.cov()` nach).

**Das ist ein kleines, aber wichtiges Beispiel dafür, dass wir die Daten, deren Eigenschaften im Detail und die Ergebnisse all der schönen, so einfach mal eben einsetzbaren Verfahren stets genauer prüfen sollten.**

Die *Normalisierung, Normierung* bzw. *Standardisierung* (werden in der Literatur synonym verwendet) ist eine Möglichkeit, mit solchen Verzerrungen umzugehen. Eine Normierung subtrahiert den Mittelwert von allen Variablen bzw. deren Werten und teilt das Ergebnis durch die Standardabweichung. So bekommen alle Variablen die gleiche Relevanz bezüglich ihrer Informationen.

Mathematisch bedeutet das, dass wir die Hauptkomponenten als Eigenvektoren nicht aus der *Kovarianz*matrix bestimmen (s. o.), sondern aus der *Korrelations*matrix, die ja relative Zusammenhänge angibt. Praktisch ergänzen wir in der PCA einfach `preprocessing.scale` auf die Daten, die wir `fit` übergeben:

In [ ]:
pca_normalized = PCA()
pca_normalized.fit(preprocessing.scale(cereals_df.iloc[:, 3:].dropna(axis=0)))
pca_normalized_summary = pcaSummary(pca_normalized)
pca_normalized_summary = pca_normalized_summary.transpose()
pca_normalized_summary.columns = [f'PC{i}' for i in range(1, len(pca_normalized_summary.columns)+1)]
pca_normalized_summary.round(4)

Nun benötigen wir sieben Komponenten, um über 90% der Informationen zu erhalten. Die Gewichte sind aber balancierter:

In [ ]:
pca_normalized_components = pd.DataFrame(pca_normalized.components_.transpose(),
                                         columns=pca_normalized_summary.columns,
                                         index=cereals_df.iloc[:, 3:].columns)
pca_normalized_components

Die erste Komponente z. B. wägt nun ab zwischen Kalorien und Portionen (cups) auf der einen Seite mit negativen Gewichten, und Proteinen, Ballaststoffen (fiber), Kalium und der Produktbewertung mit positiven Gewichten auf der anderen Seite.

Es gibt keine formal strikten Regeln, wann wir Daten normalisieren sollten. Für eine Heuristik stellen sich z. B. folgende Fragen:
* Sind die Daten in der gleichen Einheit erfasst? Euro, Kilogramm, Liter, Stunden etc.?
* Entsprechen die Skalierungen der Relevanz der Daten? Bei Beträgen z. B. große Diskrepanz zwischen Gewinn pro Aktie und Gesamtumsatz, bei zeitlichen Abläufen z. B. Millisekunden gegenüber Monaten.

## Aufgabe

Führen Sie die PCA *auf Basis von Eigenvektoren* für das Beispiel Kalorien und Bewertung *in normierter Form* durch. Stellen Sie die einzelnen Schritte wie oben dar. Hilfe zum Vorgehen:

1. Wie gewohnt die Daten einlesen, dann aber mit `preprocessing.scale` die Daten normieren.
2. Die normierten Daten von *calories* und *rating* mit ihren Eigenvektoren darstellen.
3. Mit einer `PCA` und `fit` prüfen, ob die Eigenvektoren aus 2 stimmen (und sich im Graphen nicht durch die Richtungen irritieren lassen :-)